In [3]:
import folium
import pandas as pd

In [4]:
from folium.plugins import MarkerCluster, MousePosition
from folium.features import DivIcon

In [9]:
df = pd.read_csv('/content/dataset_part_2.csv')

In [10]:
df.columns

Index(['rocket', 'payloads', 'launchpad', 'cores', 'date_utc', 'payload_mass',
       'orbit', 'booster_version', 'launch_site', 'longitude', 'latitude',
       'core_block', 'core_reused_count', 'core_serial', 'core_outcome',
       'core_flights', 'core_gridfins', 'core_reused', 'core_legs',
       'core_landing_pad', 'flight_number', 'landing_class'],
      dtype='object')

In [27]:
spacex_df = df[['launch_site', 'latitude', 'longitude', 'landing_class']]

In [28]:
launch_sites_df = spacex_df.groupby(['launch_site'], as_index=False).first()
launch_sites_df = launch_sites_df[['launch_site', 'latitude', 'longitude']]
launch_sites_df

,launch_site,latitude,longitude
0,CCSFS SLC 40,28.561857,-80.577366
1,KSC LC 39A,28.608058,-80.603956
2,VAFB SLC 4E,34.632093,-120.610829


In [29]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [30]:
circle = folium.Circle(
    nasa_coordinate,
    radius=1000,  # Radius in meters
    color='#d35400',  # Border color (orange)
    fill=True,  # Enable fill
).add_child(folium.Popup('NASA Johnson Space Center'))

marker = folium.map.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
    )
)

site_map.add_child(circle)
site_map.add_child(marker)


In [31]:
# Initialize a map centered roughly around the middle of all points
map_center = [launch_sites_df['latitude'].mean(), launch_sites_df['longitude'].mean()]
site_map = folium.Map(location=map_center, zoom_start=5)

# Add markers for each launch site
for index, row in launch_sites_df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['launch_site'],  # Show the site name on click
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(site_map)

# Display the map
site_map

In [32]:
spacex_df

,launch_site,latitude,longitude,landing_class
0,CCSFS SLC 40,28.561857,-80.577366,0
1,CCSFS SLC 40,28.561857,-80.577366,0
2,CCSFS SLC 40,28.561857,-80.577366,0
3,VAFB SLC 4E,34.632093,-120.610829,0
4,CCSFS SLC 40,28.561857,-80.577366,0
...,...,...,...,...
85,KSC LC 39A,28.608058,-80.603956,1
86,KSC LC 39A,28.608058,-80.603956,1
87,KSC LC 39A,28.608058,-80.603956,1
88,CCSFS SLC 40,28.561857,-80.577366,1


In [36]:
# Initialize a map
map_center = [spacex_df['latitude'].mean(), spacex_df['longitude'].mean()]
site_map = folium.Map(location=map_center, zoom_start=6)

# Loop through each row of the DataFrame
for index, row in spacex_df.iterrows():
    # Check if the 'landing_class' value for the current row is 1 (success)
    color = 'green' if row['landing_class'] == 1 else 'red'

    # Add a CircleMarker to the map with the appropriate color
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=10,
        color=color,
        fill=True,
        fill_color=color,
        popup=row['launch_site']  # Add launch site name as popup
    ).add_to(site_map)

site_map

In [37]:
map_center = [spacex_df['latitude'].mean(), spacex_df['longitude'].mean()]
site_map = folium.Map(location=map_center, zoom_start=5)

# Create a MarkerCluster object to hold the markers
marker_cluster = MarkerCluster().add_to(site_map)

# Loop through each row of the DataFrame
for index, row in spacex_df.iterrows():
    # Check if the 'landing_class' value for the current row is 1 (success)
    color = 'green' if row['landing_class'] == 1 else 'red'

    # Add a CircleMarker to the MarkerCluster
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=10,
        color=color,
        fill=True,
        fill_color=color,
        popup=row['launch_site']  # Add launch site name as popup
    ).add_to(marker_cluster)

# Display the map
site_map


In [57]:
import folium
from folium.plugins import MarkerCluster
from geopy.distance import geodesic

# Assuming `spacex_df` is already defined with the appropriate columns

# Initialize the map centered on the average of the launch sites
map_center = [spacex_df['latitude'].mean(), spacex_df['longitude'].mean()]
site_map = folium.Map(location=map_center, zoom_start=5)

# Create a MarkerCluster object to hold the markers
marker_cluster = MarkerCluster().add_to(site_map)

# Loop through each row of the DataFrame to add CircleMarkers to the map
for index, row in spacex_df.iterrows():
    # Check if the 'landing_class' value for the current row is 1 (success)
    color = 'green' if row['landing_class'] == 1 else 'red'

    # Add a CircleMarker to the MarkerCluster
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=10,
        color=color,
        fill=True,
        fill_color=color,
        popup=row['launch_site']  # Add launch site name as popup
    ).add_to(marker_cluster)

# Function to handle clicks and calculate distances to launch sites
def on_click(location):
    clicked_coords = [location['latlng']['lat'], location['latlng']['lng']]

    # Add a marker where the user clicked
    folium.Marker(clicked_coords, popup="Your Selected Location", icon=folium.Icon(color='green')).add_to(site_map)

    # Calculate and draw lines to each launch site and display distance
    for index, row in spacex_df.iterrows():
        launch_coords = [row['latitude'], row['longitude']]

        # Calculate the distance to the clicked location using geodesic
        distance = geodesic(clicked_coords, launch_coords).km

        # Draw a line between the clicked location and the launch site
        folium.PolyLine([clicked_coords, launch_coords], color='blue', weight=2.5, opacity=1).add_to(site_map)

        # Display distance as a label in the middle of the line
        distance_text = f"{row['launch_site']} Distance: {distance:.2f} km"
        folium.Marker(
            location=[(clicked_coords[0] + row['latitude']) / 2,
                      (clicked_coords[1] + row['longitude']) / 2],
            popup=distance_text,
            icon=folium.Icon(icon='cloud')
        ).add_to(site_map)

# Add the ClickForMarker plugin to the map
folium.ClickForMarker(popup='Selected location').add_to(site_map)

# Add the ClickForMarker plugin to the map
folium.ClickForMarker(popup='Selected location').add_to(site_map)
# Add the MousePosition plugin to the map to show coordinates on hover
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=5,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)

# Display the map
site_map


In [58]:
from geopy.distance import geodesic

# Coordinates for the given points
points = [
    (34.63165, -120.62611),
    (28.6116, -80.59724),
    (28.56225, -80.56783)
]

# Launch sites with their coordinates (assuming you have launch site data)
launch_sites = [
    {"name": "CCSFS SLC 40", "latitude": 28.561857, "longitude": -80.577366},
    {"name": "KSC LC 39A", "latitude": 28.608058, "longitude": -80.603956},
    {"name": "VAFB SLC 4E", "latitude": 34.632093, "longitude": -120.61082}
]

# Function to calculate the distance between a point and the nearest launch site
def nearest_launch_site(point, launch_sites):
    closest_site = None
    min_distance = float('inf')

    for site in launch_sites:
        site_coords = (site['latitude'], site['longitude'])
        distance = geodesic(point, site_coords).km  # Calculate distance in km

        if distance < min_distance:
            min_distance = distance
            closest_site = site['name']

    return closest_site, min_distance

# Calculate nearest launch site for each point
for point in points:
    site, distance = nearest_launch_site(point, launch_sites)
    print(f"Point {point} is closest to {site} with a distance of {distance:.2f} km.")


Point (34.63165, -120.62611) is closest to VAFB SLC 4E with a distance of 1.40 km.
Point (28.6116, -80.59724) is closest to KSC LC 39A with a distance of 0.77 km.
Point (28.56225, -80.56783) is closest to CCSFS SLC 40 with a distance of 0.93 km.
